In [ ]:
!pip install bitarray

import math
from struct import pack
from bitarray import bitarray
import numpy as np
import cv2
import os

N = 8 #block size

def dct_coefficient(input_list):
    n = len(input_list)
    if n == 1:
        return list(input_list)
    else:
        half = n // 2
        alpha = [(input_list[i] + input_list[-(i + 1)]) for i in range(half)]
        beta = [(input_list[i] - input_list[-(i + 1)]) / (math.cos((i + 0.5) * math.pi / n) * 2.0) for i in range(half)]
        alpha = dct_coefficient(alpha)
        beta = dct_coefficient(beta)
        result = []
        for i in range(half - 1):
            result.append(alpha[i])
            result.append(beta[i] + beta[i + 1])
        result.append(alpha[-1])
        result.append(beta[-1])
        return result




def dct2_coefficients(matrix):
    size = len(matrix)
    A = [[0 for _ in range(size)] for __ in range(size)]

    scaling = math.sqrt(2 / size)

    for col in range(size):
        cur_column = []
        for row in range(size):
            cur_column.append(matrix[row][col])
        cur_column = dct_coefficient(cur_column)
        for row in range(size):
            A[row][col] = cur_column[row] * scaling
            if row == 0:
                A[row][col] /= math.sqrt(2)

    for row in range(size):
        cur_row = []
        for col in range(size):
            cur_row.append(A[row][col])
        cur_row = dct_coefficient(cur_row)
        for col in range(size):
            A[row][col] = cur_row[col] * scaling
            if col == 0:
                A[row][col] /= math.sqrt(2)

    return A


def quantize_matrix(matrix, quantization_matrix):
    size = len(matrix)
    quantized_matrix = [[0 for _ in range(size)] for __ in range(size)]

    for row in range(size):
        for col in range(size):
            quantized_matrix[row][col] = round(matrix[row][col] / quantization_matrix[row][col])

    return quantized_matrix



def zigzag_order(matrix):
    size = len(matrix)
    index = 0
    go_down = False
    zigzag = [0 for _ in range(size * size)]

    for summation in range(2 * size - 1):
        row, col = None, None

        if go_down:
            row = 0
            col = summation - row
            if col >= size:
                col = size - 1
                row = summation - col
        else:
            col = 0
            row = summation - col
            if row >= size:
                row = size - 1
                col = summation - row

        while row >= 0 and row < size and col >= 0 and col < size:
            zigzag[index] = matrix[row][col]
            index += 1
            if go_down:
                row += 1
                col -= 1
            else:
                row -= 1
                col += 1
        go_down = not go_down

    return zigzag





def run_length_encode(zigzag):
    start, end = 1, 1
    last_index = N*N-1
    encoded_result = []

    while last_index >= 0 and zigzag[last_index] == 0:
        last_index -= 1

    while end <= last_index:
        if (zigzag[end] == 0 and end - start + 1 == 16) or zigzag[end] != 0:
            encoded_result += [end - start, int(zigzag[end]).bit_length(), zigzag[end]]
            start = end + 1
        end += 1

    if last_index != N*N-1:
        encoded_result += [0, 0]

    return encoded_result



def huffman_code(code_dict, node, code):

    if node[1][1] is None and node[1][2] is None:
        code_dict[node[1][0]] = code
        return

    if node[1][1] is not None:
        code_next = code + "0"
        huffman_code(code_dict, node[1][1], code_next)

    if node[1][2] is not None:
        code_next = code + "1"
        huffman_code(code_dict, node[1][2], code_next)



def modify_huffman_code(code_dict):
    new_code_dict = dict()
    values_list = [[] for _ in range(17)]

    for (key, val) in code_dict.items():
        values_list[len(val)].append(key)

    current_value = 0
    for i in range(17):
        for j in range(len(values_list[i])):
            current_binary = bin(current_value)[2:]
            if '0' * (i - len(current_binary)) + current_binary == '1' * i:
                values_list[i + 1] = [values_list[i][j]] + values_list[i + 1]
                continue
            new_code_dict[values_list[i][j]] = '0' * (i - len(current_binary)) + current_binary
            current_value += 1
        current_value <<= 1

    return new_code_dict




def rgb_to_ycbcr(red, green, blue):
    y = [[0.299 * red[i][j] + 0.587 * green[i][j] + 0.114 * blue[i][j] for j in range(N)] for i in range(N)]
    cb = [[-0.169 * red[i][j] - 0.331 * green[i][j] + 0.5 * blue[i][j] for j in range(N)] for i in range(N)]
    cr = [[0.5 * red[i][j] - 0.419 * green[i][j] - 0.081 * blue[i][j] for j in range(N)] for i in range(N)]
    return y, cb, cr


def norm_coefficient(index):
    if index == 0:
        return 1.0 / math.sqrt(2.0)
    else:
        return 1.0



import heapq

def huffman_code_generator(input_string):
    frequency_dict = dict()

    for i in range(len(input_string)):
        if input_string[i] in frequency_dict:
            frequency_dict[input_string[i]] += 1
        else:
            frequency_dict[input_string[i]] = 1

    priority_queue = list()

    for element in frequency_dict:
        priority_queue.append([frequency_dict[element], [element, None, None]])

    priority_queue.sort()

    while len(priority_queue) > 1:
        t1 = priority_queue[0]
        t2 = priority_queue[1]

        current_node = [t1[0] + t2[0], [-1, t1, t2]]
        priority_queue.remove(t1)
        priority_queue.remove(t2)
        priority_queue.append(current_node)
        priority_queue.sort(key=lambda t: t[0])

    huffman_tree = priority_queue[0]

    code_dict = dict()
    code = ""
    huffman_code(code_dict, huffman_tree, code)
    code_dict = modify_huffman_code(code_dict)
    return code_dict


from struct import pack

def write_quantization_table(jpeg_out, quantization_table, table_type):
    jpeg_out.write(b'\xff\xdb')  # marker
    jpeg_out.write(pack(">H", 67))  # length of chunk
    jpeg_out.write(pack("B", table_type))
    zigzag_quantization = zigzag_order(quantization_table)

    for i in range(N*N):
        jpeg_out.write(pack("B", zigzag_quantization[i]))



from struct import pack

def write_start_of_frame(jpeg_out, height, width, components):
    jpeg_out.write(b'\xff\xc0')  # marker
    jpeg_out.write(pack(">H", 8 + 3 * components))  # length of chunk
    jpeg_out.write(pack("B", 8))  # bits/sample
    jpeg_out.write(pack(">HHB", height, width, components))  # height, width, components of image

    for i in range(components):
        # id, sampling factor, quantization table id
        jpeg_out.write(pack("BBB", i + 1, 16 + 1, min(i, 1)))



from struct import pack

def write_huffman_table(jpeg_out, huffman_type, huffman_code):
    jpeg_out.write(b'\xff\xc4')  # marker
    jpeg_out.write(pack(">HB", 19 + len(huffman_code), huffman_type))

    value_list = [[] for _ in range(16)]
    for (key, val) in huffman_code.items():
        value_list[len(val) - 1].append(key)

    for i in range(16):
        jpeg_out.write(pack("B", len(value_list[i])))

    for i in range(16):
        for value in value_list[i]:
            jpeg_out.write(pack("B", value))



def add_FF00_sequence(data):
    result = ""
    for i in range(0, len(data), 8):
        current_byte = data[i:i+8]
        result = result + current_byte
        if current_byte == "11111111":
            result = result + "00000000"
    return result


from struct import pack
from bitarray import bitarray

def write_start_of_scan(jpeg_out, components, Y_dc_list, CB_dc_list, CR_dc_list, Y_ac_list, CB_ac_list, CR_ac_list,
                        Y_dc_vli_list, CB_dc_vli_list, CR_dc_vli_list, Y_ac_vli_list, CB_ac_vli_list, CR_ac_vli_list,
                        lum_dc_code, lum_ac_code, chr_dc_code, chr_ac_code):
    jpeg_out.write(b'\xff\xda')
    jpeg_out.write(pack(">HB", 6 + 2 * components, components))
    jpeg_out.write(b'\x01\x00\x02\x11\x03\x11')
    jpeg_out.write(pack("BBB", 0, 63, 0))
    Y_dc_ptr, CB_dc_ptr, CR_dc_ptr, Y_ac_ptr, CB_ac_ptr, CR_ac_ptr,Y_dc_vli_ptr, CB_dc_vli_ptr, CR_dc_vli_ptr, Y_ac_vli_ptr, CB_ac_vli_ptr, CR_ac_vli_ptr = 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0
    data = ""
    while Y_dc_ptr < len(Y_dc_list):
        data = data + lum_dc_code[Y_dc_list[Y_dc_ptr]] + Y_dc_vli_list[Y_dc_vli_ptr]
        Y_dc_ptr += 1
        Y_dc_vli_ptr += 1
        length_block = 1
        while length_block < N*N and Y_ac_list[Y_ac_ptr] != 0:
            data = data + lum_ac_code[Y_ac_list[Y_ac_ptr]] + Y_ac_vli_list[Y_ac_vli_ptr]

            length_block += (Y_ac_list[Y_ac_ptr] >> 4)
            length_block += 1
            Y_ac_ptr += 1
            Y_ac_vli_ptr += 1
        if length_block < N*N:
            data = data + lum_ac_code[0]
            Y_ac_ptr += 1

        data = data + chr_dc_code[CB_dc_list[CB_dc_ptr]] + CB_dc_vli_list[CB_dc_vli_ptr]
        CB_dc_ptr += 1
        CB_dc_vli_ptr += 1
        length_block = 1
        while length_block < N*N and CB_ac_list[CB_ac_ptr] != 0:
            data = data + chr_ac_code[CB_ac_list[CB_ac_ptr]] + CB_ac_vli_list[CB_ac_vli_ptr]
            length_block += (CB_ac_list[CB_ac_ptr] >> 4)
            length_block += 1
            CB_ac_ptr += 1
            CB_ac_vli_ptr += 1
        if length_block < N*N:
            data = data + chr_ac_code[0]
            CB_ac_ptr += 1

        data = data + chr_dc_code[CR_dc_list[CR_dc_ptr]] + CR_dc_vli_list[CR_dc_vli_ptr]
        CR_dc_ptr += 1
        CR_dc_vli_ptr += 1
        length_block = 1
        while length_block < N*N and CR_ac_list[CR_ac_ptr] != 0:
            data = data + chr_ac_code[CR_ac_list[CR_ac_ptr]] + CR_ac_vli_list[CR_ac_vli_ptr]
            length_block += (CR_ac_list[CR_ac_ptr] >> 4)
            length_block += 1
            CR_ac_ptr += 1
            CR_ac_vli_ptr += 1
        if length_block < N*N:
            data = data + chr_ac_code[0]
            CR_ac_ptr += 1

    while len(data) % 8 != 0:
        data = data + "0"
    data = add_FF00_sequence(data)
    data = bitarray(data)
    jpeg_out.write(data)


def get_variable_length_integer(x):
    if x == 0:
        return ''
    if x < 0:
        x = -x
        binary_representation = bin((~x) & ((1 << int(x).bit_length()) - 1))[2:]
        binary_representation = "0" * (len(bin(x)) - 2 - len(binary_representation)) + binary_representation
    else:
        binary_representation = bin(x)[2:]
    return binary_representation

def run_length_encoding_to_bits(rle):
    rl_class, vli = [], []
    for i in range(0, len(rle), 3):
        rl_class.append(((rle[i] << 4) + rle[i + 1]))
        if rl_class[-1] == 0:
            break
        vli.append(get_variable_length_integer(rle[i + 2]))
    return rl_class, vli


def pad_image(r, g, b):
    new_height, new_width = ((len(r) + 7) // 8) * 8, ((len(r[0]) + 7) // 8) * 8
    new_r = [[r[min(i, len(r) - 1)][min(j, len(r[0]) - 1)] for j in range(new_width)] for i in range(new_height)]
    new_g = [[g[min(i, len(g) - 1)][min(j, len(g[0]) - 1)] for j in range(new_width)] for i in range(new_height)]
    new_b = [[b[min(i, len(b) - 1)][min(j, len(b[0]) - 1)] for j in range(new_width)] for i in range(new_height)]

    return new_r, new_g, new_b


def encode(input_r, input_g, input_b):
    height, width = len(input_r), len(input_r[0])
    padded_r, padded_g, padded_b = pad_image(input_r, input_g, input_b)
    Y_dc_list, CB_dc_list, CR_dc_list = [], [], []
    Y_ac_list, CB_ac_list, CR_ac_list = [], [], []
    Y_dc_vli_list, CB_dc_vli_list, CR_dc_vli_list = [], [], []
    Y_ac_vli_list, CB_ac_vli_list, CR_ac_vli_list = [], [], []

    quant_lum = [
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 48, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ]
    quant_chr = [
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99, 99],
        [24, 26, 56, 99, 99, 99, 99, 99],
        [47, 66, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99]
    ]

    scale = float(input("scaling factor "))
    quant_lum = [[min(max(1, int(quant_lum[i][j] * scale)), 255) for j in range(N)] for i in range(N)]
    quant_chr = [[min(max(1, int(quant_chr[i][j] * scale)), 255) for j in range(N)] for i in range(N)]

    prev_Y_dc, prev_CB_dc, prev_CR_dc = 0, 0, 0
    for i in range(len(padded_r) // 8):
        for j in range(len(padded_r[0]) // 8):
            cur_r = [[padded_r[x][y] - 128 for y in range(j * 8, j * 8 + 8)] for x in range(i * 8, i * 8 + 8)]
            cur_g = [[padded_g[x][y] - 128 for y in range(j * 8, j * 8 + 8)] for x in range(i * 8, i * 8 + 8)]
            cur_b = [[padded_b[x][y] - 128 for y in range(j * 8, j * 8 + 8)] for x in range(i * 8, i * 8 + 8)]

            y, cb, cr = rgb_to_ycbcr(cur_r, cur_g, cur_b)
            Y, CB, CR = dct2_coefficients(y), dct2_coefficients(cb), dct2_coefficients(cr)

            Y, CB, CR = quantize_matrix(Y, quant_lum), quantize_matrix(CB, quant_chr), quantize_matrix(CR, quant_chr)
            Y_z, CB_z, CR_z = zigzag_order(Y), zigzag_order(CB), zigzag_order(CR)

            Y_rle, CB_rle, CR_rle = run_length_encode(Y_z), run_length_encode(CB_z), run_length_encode(CR_z)
            Y_rl_class, Y_vli = run_length_encoding_to_bits(Y_rle)
            CB_rl_class, CB_vli = run_length_encoding_to_bits(CB_rle)
            CR_rl_class, CR_vli = run_length_encoding_to_bits(CR_rle)

            Y_ac_list = Y_ac_list + Y_rl_class
            CB_ac_list = CB_ac_list + CB_rl_class
            CR_ac_list = CR_ac_list + CR_rl_class

            Y_ac_vli_list = Y_ac_vli_list + Y_vli
            CB_ac_vli_list = CB_ac_vli_list + CB_vli
            CR_ac_vli_list = CR_ac_vli_list + CR_vli

            Y_dc, CB_dc, CR_dc = Y_z[0] - prev_Y_dc, CB_z[0] - prev_CB_dc, CR_z[0] - prev_CR_dc
            Y_dc_list.append(int(Y_dc).bit_length())
            CB_dc_list.append(int(CB_dc).bit_length())
            CR_dc_list.append(int(CR_dc).bit_length())

            Y_dc_vli_list.append(get_variable_length_integer(Y_dc))
            CB_dc_vli_list.append(get_variable_length_integer(CB_dc))
            CR_dc_vli_list.append(get_variable_length_integer(CR_dc))

            prev_Y_dc, prev_CB_dc, prev_CR_dc = Y_z[0], CB_z[0], CR_z[0]

    lum_dc_code = huffman_code_generator(Y_dc_list)
    lum_ac_code = huffman_code_generator(Y_ac_list)
    chr_dc_code = huffman_code_generator(CB_dc_list + CR_dc_list)
    chr_ac_code = huffman_code_generator(CB_ac_list + CR_ac_list)

    output_file = input("output image name :  ")
    jpeg_out = open(output_file + ".jpg", "wb")
    jpeg_out.write(b'\xff\xd8\xff\xe0')
    jpeg_out.write(pack(">H", 16))
    jpeg_out.write(b'JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00')

    write_quantization_table(jpeg_out, quant_lum, 0)
    write_quantization_table(jpeg_out, quant_chr, 1)

    write_start_of_frame(jpeg_out, height, width, 3)

    write_huffman_table(jpeg_out, 0, lum_dc_code)
    write_huffman_table(jpeg_out, 16, lum_ac_code)
    write_huffman_table(jpeg_out, 1, chr_dc_code)
    write_huffman_table(jpeg_out, 17, chr_ac_code)

    write_start_of_scan(jpeg_out, 3, Y_dc_list, CB_dc_list, CR_dc_list, Y_ac_list, CB_ac_list, CR_ac_list,
              Y_dc_vli_list, CB_dc_vli_list, CR_dc_vli_list, Y_ac_vli_list, CB_ac_vli_list, CR_ac_vli_list,
              lum_dc_code, lum_ac_code, chr_dc_code, chr_ac_code)

    jpeg_out.write(b'\xff\xd9')
    jpeg_out.close()



file = input("path of the image ")
img = cv2.imread(file)
r = img[:,:,2]
r = [[r[i][j] for j in range(len(r[0]))] for i in range(len(r))]
g = img[:,:,1]
g = [[g[i][j] for j in range(len(g[0]))] for i in range(len(g))]
b = img[:,:,0]
b = [[b[i][j] for j in range(len(b[0]))] for i in range(len(b))]

encode(r,g,b)




def display_image_size(file):
    size = os.path.getsize(file)
    print(f"The size of the image file is: {size} bytes")

display_image_size(file)






path of the image /content/cameraman.jpg
scaling factor 30
output image name :  hdsghjfdg
The size of the image file is: 132986 bytes


In [ ]:
import os
file = input("path of the image for output image ")
img = cv2.imread(file)


def display_output_size(file):
    size = os.path.getsize(file)
    print(f"The size of the output file is: {size} bytes")


display_output_size(file)

path of the image for output image /content/s40.jpg
The size of the output file is: 4513 bytes
